# Ollama

[Ollama](https://ollama.ai/) allows you to run open-source large language models, such as Llama 2, locally.

Ollama bundles model weights, configuration, and data into a single package, defined by a Modelfile. 

It optimizes setup and configuration details, including GPU usage.

For a complete list of supported models and model variants, see the [Ollama model library](https://ollama.ai/library).

## Setup

First, follow [these instructions](https://github.com/jmorganca/ollama) to set up and run a local Ollama instance:

* [Download](https://ollama.ai/download)
* Fetch a model via `ollama pull <model family>`
* e.g., for `Llama-7b`: `ollama pull llama2`
* This will download the most basic version of the model typically (e.g., smallest # parameters and `q4_0`)
* On Mac, it will download to 

`~/.ollama/models/manifests/registry.ollama.ai/library/<model family>/latest`

* And we specify a particular version, e.g., for `ollama pull vicuna:13b-v1.5-16k-q4_0`
* The file is here with the model version in place of `latest`

`~/.ollama/models/manifests/registry.ollama.ai/library/vicuna/13b-v1.5-16k-q4_0`

You can easily access models in a few ways:

1/ if the app is running:
* All of your local models are automatically served on `localhost:11434`
* Select your model when setting `llm = Ollama(..., model="<model family>:<version>")`
* If you set `llm = Ollama(..., model="<model family")` withoout a version it will simply look for `latest`

2/ if building from source or just running the binary: 
* Then you must run `ollama serve`
* All of your local models are automatically served on `localhost:11434`
* Then, select as shown above


## Usage

You can see a full list of supported parameters on the [API reference page](https://api.python.langchain.com/en/latest/llms/langchain.llms.ollama.Ollama.html).

If you are using a LLaMA `chat` model (e.g., `ollama pull llama2:7b-chat`) then you can use the `ChatOllama` interface.

This includes [special tokens](https://huggingface.co/blog/llama2#how-to-prompt-llama-2) for system message and user input.

In [2]:
from langchain.chat_models import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler                                  
chat_model = ChatOllama(base_url="http://localhost:11434", 
             model="llama2:7b-chat", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

With `StreamingStdOutCallbackHandler`, you will see tokens streamed.

In [3]:
from langchain.schema import HumanMessage

messages = [
    HumanMessage(content="Tell me about the history of AI")
]
chat_model(messages)

 Artificial intelligence (AI) has a rich and varied history that spans several decades. Hinweis: This is a brief overview of some of the key milestones in the history of AI. For a more detailed account, I recommend checking out some of the many books on the subject or visiting a reputable online resource.

1. Early Ideas and Concepts (1950s-1960s): The concept of artificial intelligence can be traced back to ancient Greece, where myths spoke of artificial creatures crafted by humans. However, the modern field of AI began to take shape in the 1950s with the work of computer scientists like Alan Turing and Marvin Minsky. They proposed the idea of a machine that could think and learn like a human, leading to the development of the first AI programs.
2. Rule-Based Systems (1970s-1980s): In the 1970s and 1980s, AI research focused on developing rule-based systems, which used pre-defined rules to reason and make decisions. This led to the development of expert systems, which were designed to

AIMessageChunk(content=' Artificial intelligence (AI) has a rich and varied history that spans several decades. Hinweis: This is a brief overview of some of the key milestones in the history of AI. For a more detailed account, I recommend checking out some of the many books on the subject or visiting a reputable online resource.\n\n1. Early Ideas and Concepts (1950s-1960s): The concept of artificial intelligence can be traced back to ancient Greece, where myths spoke of artificial creatures crafted by humans. However, the modern field of AI began to take shape in the 1950s with the work of computer scientists like Alan Turing and Marvin Minsky. They proposed the idea of a machine that could think and learn like a human, leading to the development of the first AI programs.\n2. Rule-Based Systems (1970s-1980s): In the 1970s and 1980s, AI research focused on developing rule-based systems, which used pre-defined rules to reason and make decisions. This led to the development of expert syst

## RAG

We can use Olama with RAG, [just as shown here](https://python.langchain.com/docs/use_cases/question_answering/how_to/local_retrieval_qa).

Let's use the 13b model:

```
ollama pull llama2:13b
ollama run llama2:13b 
```

Let's also use local embeddings from `GPT4AllEmbeddings` and `Chroma`.

In [ ]:
! pip install gpt4all chromadb

In [4]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

Found model file at  /Users/rlm/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


In [7]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [8]:
from langchain import PromptTemplate

# Prompt
template = """[INST] <<SYS>> Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. <</SYS>>
{context}
Question: {question}
Helpful Answer:[/INST]"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [9]:
# Chat model
from langchain.chat_models import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat_model = ChatOllama(base_url="http://localhost:11434",
             model="llama2:7b-chat",
             verbose=True,
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [10]:
# QA chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    chat_model,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [11]:
question = "What are the various approaches to Task Decomposition for AI Agents?"
result = qa_chain({"query": question})

 There are three main approaches to task decomposition for AI agents: (1) using simple prompts like "Steps for XYZ." and "What are the subgoals for achieving XYZ?", (2) using task-specific instructions, and (3) incorporating human inputs.

You can also get logging for tokens.

In [12]:
from langchain.schema import LLMResult
from langchain.callbacks.base import BaseCallbackHandler

class GenerationStatisticsCallback(BaseCallbackHandler):
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(response.generations[0][0].generation_info)
        
callback_manager = CallbackManager([StreamingStdOutCallbackHandler(), GenerationStatisticsCallback()])

chat_model = ChatOllama(base_url="http://localhost:11434",
             model="llama2",
             verbose=True,
             callback_manager=callback_manager)

qa_chain = RetrievalQA.from_chain_type(
    chat_model,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "What are the approaches to Task Decomposition?"
result = qa_chain({"query": question})

 There are three approaches to task decomposition: (1) using simple prompts like "Steps for XYZ.\n1." and "What are the subgoals for achieving XYZ?" to guide the LLM, (2) providing task-specific instructions, such as "Write a story outline." for writing a novel, and (3) incorporating human inputs.{'model': 'llama2', 'created_at': '2023-08-23T04:11:15.097541Z', 'done': True, 'context': [1, 29871, 1, 29961, 25580, 29962, 518, 25580, 29962, 518, 25580, 29962, 3532, 14816, 29903, 6778, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 278, 1139, 472, 278, 1095, 29889, 29871, 13, 3644, 366, 1016, 29915, 29873, 1073, 278, 1234, 29892, 925, 1827, 393, 366, 1016, 29915, 29873, 1073, 29892, 1016, 29915, 29873, 1018, 304, 1207, 701, 385, 1234, 29889, 29871, 13, 11403, 2211, 25260, 7472, 322, 3013, 278, 1234, 408, 3022, 895, 408, 1950, 29889, 529, 829, 14816, 29903, 6778, 13, 5398, 26227, 508, 367, 2309, 313, 29896, 29897, 491, 365, 26369, 411, 2560, 9508, 292, 763, 376, 7789, 567, 363, 1060, 29979, 

`eval_count` / (`eval_duration`/10e9)  gets `tok / s`

In [13]:
79 / (1585813000/1000/1000/1000)

49.8167186168861